In [1]:
//Import para Jupyter-notebooks 
import $ivy.`org.apache.spark::spark-sql:2.4.0`
import $ivy.`sh.almond::almond-spark:0.6.0`
import org.apache.log4j.{Level, Logger}
Logger.getLogger("org").setLevel(Level.OFF)

import $ivy.$                                  

import $ivy.$                              

import org.apache.log4j.{Level, Logger}


---
## Funciones de agregación más utilizadas

1. Definir la sesión de Spark

In [2]:
import org.apache.spark.sql._

val spark = {
  NotebookSparkSession.builder()
    .master("local[*]")
    .getOrCreate()
}

Loading spark-stubs
Getting spark JARs
Creating SparkSession


Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties


Spark UI

import org.apache.spark.sql._


spark: SparkSession = org.apache.spark.sql.SparkSession@1fa7409a

2. Crear un dataframe desde un archivo plano ubicación: resources/Impuesto_de_Transporte_-_MinEnerg_a.csv 

In [ ]:
import spark.implicits._
val impuestoDf = ???

In [3]:
import spark.implicits._
val impuestoDf = spark.read.format("csv").option("header",true).load("resources/Impuesto_de_Transporte_-_MinEnerg_a.csv")

load at cmd2.sc:2

1 / 1

import spark.implicits._

impuestoDf: DataFrame = [ano: string, trimestre: string ... 6 more fields]

3. Cálcula cuantos elementos tiene el dataframe, pero que no sea una acción

In [ ]:
import org.apache.spark.sql.functions.count
impuestoDf.select(count("ano")).show() 

4. Ahora cálcula la suma del totalimpuestos pagados durante el año 2019 trimestre 1

In [ ]:
import org.apache.spark.sql.functions.sum


In [ ]:
// dataframes
import org.apache.spark.sql.functions.sum
impuestoDf.where("ano = 2019 and trimestre = 1").select(sum("totalimpuesto").as("total")).show()
// SQL
impuestoDf.createOrReplaceTempView("impuestos")
spark.sql("select sum(totalimpuesto) as total from impuestos where ano = 2019 and trimestre = 1").show()

5. Calcula el total de impuestos pagos en el año 2019 trimestre 1, el total pagado, el promedio pagadao , el menor impuesto y el mayor impuesto

In [ ]:
import org.apache.spark.sql.functions.{sum, count, avg, min, max}

val totals = ???
totals.show()


In [ ]:
import org.apache.spark.sql.functions.{sum, count, avg, min, max}

val totals = impuestoDf.where("ano = 2019 and trimestre = 1").select(
    count("totalimpuesto").alias("numImpuestos"),
    sum("totalimpuesto").as("totalPagado"),
    avg("totalimpuesto").as("promedioPagado"),
    max("totalimpuesto").as("MayorTotalPagado"),
    min("totalimpuesto").as("MenorTotalPagado")    
)

totals.show()
// esta función sirve para describir una columna, obtiene el count, mean, desviación estandar, min y max.
impuestoDf.select("totalimpuesto").describe().show()

6. Agrupa la información por departament, primero con la función de los dataframes

In [ ]:
impuestoDf._____

In [ ]:
impuestoDf.groupBy('departamento).count().orderBy('count).show()

6.b. Ahora obten el mismo resultado ejecutando la sentencia SQL

In [ ]:
_______('impuestosTable)


In [4]:
impuestoDf.createOrReplaceTempView("impuestosTable")
spark.sql("SELECT departamento,count(1) as count FROM impuestosTable GROUP BY departamento ORDER BY count").show()

show at cmd3.sc:2

1 / 1

show at cmd3.sc:2

200 / 200

+------------------+-----+
|      departamento|count|
+------------------+-----+
|   DEPARTAMENTO NN|   12|
|             CAUCA|   14|
|            CALDAS|   18|
|      CUNDINAMARCA|   48|
|            ARAUCA|   54|
|              META|   68|
|          PUTUMAYO|  102|
|             HUILA|  108|
|         MAGDALENA|  112|
|          CASANARE|  128|
|           BOLIVAR|  140|
|         ANTIOQUIA|  142|
|             CESAR|  144|
|           CORDOBA|  170|
|            NARIÑO|  180|
|         SANTANDER|  184|
|NORTE DE SANTANDER|  192|
|             SUCRE|  201|
|            TOLIMA|  335|
|            BOYACA|  391|
+------------------+-----+



7. Calcula el totalimpuesto cancelado por cada departamento ordenado desc

In [ ]:
//SQL 
spark.sql("""
SELECT departamento,sum(totalimpuesto) as total 
FROM impuestosTable GROUP BY departamento ORDER BY total desc
""").show()